In [19]:
import os,datetime
import cv2
import xml.etree.ElementTree as ET
from PIL import Image
import numpy as np
import random
from pathlib import Path
'''
SEED_XML_DIR = '/data/darknet/python/xml/'
SEED_IMG_DIR = '/data/darknet/python/image/'
SPEC_OBJ_DIR = '../manufactur/special_obj/'
GENE_IMG_WITH_SPECIAL_OBJ = '../generated/img_with_special_obj/'
GENE_XML_WITH_SPECIAL_OBJ = '../generated/xml_with_special_obj/'
'''
SEED_IMG_DIR = '/data/wxjs/JPEGImages/'
SEED_XML_DIR = '/data/wxjs/Changed_Annotations/'
SPEC_OBJ_DIR = '/data/wxjs/generated/img_objects/'
GENE_IMG_WITH_SPECIAL_OBJ = '/data/wxjs/generated/img_with_special_obj/'
GENE_XML_WITH_SPECIAL_OBJ = '/data/wxjs/generated/xml_with_special_obj/'

def IOU(bbox_a, bbox_b):
    '''
    W = min(A.RT.x, B.RT.x) - max(A.LB.x, B.LB.x) 
    H = min(A.RT.y, B.RT.y) - max(A.LB.y, B.LB.y) 
    if W <= 0 or H <= 0: 
        return 0 
    SA = (A.RT.x - A.LB.x) * (A.RT.y - A.LB.y) 
    SB = (B.RT.x - B.LB.x) * (B.RT.y - B.LB.y) 
    cross = W * H return cross/(SA + SB - cross)
    '''
    W = min(bbox_a[1], bbox_b[1]) - max(bbox_a[0], bbox_b[0]) 
    H = min(bbox_a[3], bbox_b[3]) - max(bbox_a[2], bbox_b[2]) 
    if W <= 0 or H <= 0: 
        return 0
    SA = (bbox_a[1] - bbox_a[0]) * (bbox_a[3] - bbox_a[2]) 
    SB = (bbox_b[1] - bbox_b[0]) * (bbox_b[3] - bbox_b[2])  
    cross = W * H 
    return cross/(SA + SB - cross)

def generate_new_bbox(img_size, img_obj_size):
    array_x = np.arange(int(img_size[1] - img_obj_size[1]))
    array_y = np.arange(int(img_size[0] - img_obj_size[0]))
    random_x = random.sample(list(array_x), 1)[0]
    random_y = random.sample(list(array_y), 1)[0]
    new_position = (random_x, random_y)
    new_bbox = [random_x, random_x + img_obj_size[1], random_y, random_y + img_obj_size[0]]
    #print(new_bbox)
    return new_bbox
def get_bboxes_from_etree(etree):
    root = tree.getroot()  
    objects = root.findall('object')
    bboxes = []
    for obj in objects:
        xmlbox = obj.find('bndbox')
        b = [float(xmlbox.find('xmin').text), float(xmlbox.find('xmax').text), float(xmlbox.find('ymin').text), float(xmlbox.find('ymax').text)]
        bboxes.append(b)
    return bboxes

def check_bbox(new_bbox, bboxes):
    for bbox in bboxes:
        if(IOU(new_bbox, bbox) > 0.005):
            return False
        continue
    return True

def past_and_insert(img_obj, img_array, new_position, obj_element, etree):
    new_xmlobj = generate_new_xmlobj(obj_element, new_position, img_obj.shape[:2])
    new_xml_etree = insert_to_xml(new_xmlobj, etree)
    new_pil_img = past_obj_to_background(img_obj, img_array, new_position)
    return new_pil_img, new_xml_etree

def generate_new_xmlobj(xmlobj_old, new_position, new_size):
    element_object = ET.Element('object')
    tag_name = ET.SubElement(element_object, 'name')
    tag_name.text = xmlobj_old.find('name').text

    tag_difficult = ET.SubElement(element_object, 'difficult')
    tag_difficult.text = xmlobj_old.find('difficult').text

    element_bndbox = ET.SubElement(element_object, 'bndbox')
    tag_xmin = ET.SubElement(element_bndbox, 'xmin')
    tag_ymin = ET.SubElement(element_bndbox, 'ymin')
    tag_xmax = ET.SubElement(element_bndbox, 'xmax')
    tag_ymax = ET.SubElement(element_bndbox, 'ymax')
    tag_xmin.text = str(new_position[0])
    tag_ymin.text = str(new_position[1])
    tag_xmax.text = str(new_position[0] + new_size[1])
    tag_ymax.text = str(new_position[1] + new_size[0])
    return element_object

def insert_to_xml(xml_obj, xml_etree):
    root = xml_etree.getroot()
    root.append(xml_obj)
    return xml_etree
    
def past_obj_to_background(img_obj, img_array, position = (200, 200)):
    img = Image.fromarray(img_array)
    img_obj = Image.fromarray(img_obj)
    img.paste(img_obj, position)
    return img

def inset_obj_to_an_image_and_xml(img_obj, img, obj_element, etree):
    new_bbox = generate_new_bbox(img.shape[:2], img_obj.shape[:2])
    print(new_bbox)
    bboxes = get_bboxes_from_etree(etree)
    while(not check_bbox(new_bbox, bboxes)):
        print('new_bbox not suitable, retry...')
        new_bbox = generate_new_bbox(img.shape[:2], img_obj.shape[:2])
        print(new_bbox)
    new_pil_img, new_xml_etree = past_and_insert(img_obj, img, (new_bbox[0], new_bbox[2]), obj_element, etree)
    
    return new_pil_img, new_xml_etree

def generate_new_xmlobj(xmlobj_old, new_position, new_size):
    element_object = ET.Element('object')
    tag_name = ET.SubElement(element_object, 'name')
    tag_name.text = xmlobj_old.find('name').text

    tag_difficult = ET.SubElement(element_object, 'difficult')
    tag_difficult.text = xmlobj_old.find('difficult').text

    element_bndbox = ET.SubElement(element_object, 'bndbox')
    tag_xmin = ET.SubElement(element_bndbox, 'xmin')
    tag_ymin = ET.SubElement(element_bndbox, 'ymin')
    tag_xmax = ET.SubElement(element_bndbox, 'xmax')
    tag_ymax = ET.SubElement(element_bndbox, 'ymax')
    tag_xmin.text = str(new_position[0])
    tag_ymin.text = str(new_position[1])
    tag_xmax.text = str(new_position[0] + new_size[1])
    tag_ymax.text = str(new_position[1] + new_size[0])
    return element_object

def generate_xml_info_under_object(attribut_vale_dic):
    element_object = ET.Element('object')
    if(len(attribut_vale_dic) == 0):
        return False
    for key in attribut_vale_dic.keys():
        tag_name = ET.SubElement(element_object, key)
        tag_name.text = attribut_vale_dic[key]
    return element_object
#generate_xml_info_under_object({'name':'my-class','difficult':'0'})

In [5]:
'''
seed_xml_names = os.listdir(SEED_XML_DIR)
special_obj_cls = os.listdir(SPEC_OBJ_DIR)
#classes = ['person', 'bicycle', 'bottle', 'cup']
classes = ['slagcar_00']
for xml_name in seed_xml_names:
    img_name = SEED_IMG_DIR + xml_name[:-3] + 'jpg'
    xml_name = SEED_XML_DIR + xml_name
    print(img_name)
    print(xml_name)
    tree=ET.parse(xml_name)
    img = cv2.imread(img_name)
    for cls in special_obj_cls:
        if(not cls in classes):
            continue
        objs = os.listdir(SPEC_OBJ_DIR + cls)
        for obj in objs:
            print(obj)
            if(not (obj.endswith('.png') or obj.endswith('.jpg'))):
                continue
            img_obj = cv2.imread(SPEC_OBJ_DIR + cls + '/' + obj)
            dic_ = {'name':cls, 'difficult':'0'}
            obj_element = generate_xml_info_under_object(dic_)
            new_pil_img, new_xml_etree = inset_obj_to_an_image_and_xml(img_obj, img, obj_element, tree)
            print('insert success')
            tree = new_xml_etree
            img = np.array(new_pil_img)
    ## save xml and img
    print('save image and xml')
    time_mark = datetime.datetime.now()
    time_str = time_mark.strftime("%Y%m%d%H%M%S_")
    new_pil_img = Image.fromarray(img[:,:,(2, 1, 0)])
    new_pil_img.save(GENE_IMG_WITH_SPECIAL_OBJ + time_str + img_name.split('/')[-1])
    tree.write(GENE_XML_WITH_SPECIAL_OBJ + time_str + xml_name.split('/')[-1])
    #break
'''

'\nseed_xml_names = os.listdir(SEED_XML_DIR)\nspecial_obj_cls = os.listdir(SPEC_OBJ_DIR)\n#classes = [\'person\', \'bicycle\', \'bottle\', \'cup\']\nclasses = [\'slagcar_00\']\nfor xml_name in seed_xml_names:\n    img_name = SEED_IMG_DIR + xml_name[:-3] + \'jpg\'\n    xml_name = SEED_XML_DIR + xml_name\n    print(img_name)\n    print(xml_name)\n    tree=ET.parse(xml_name)\n    img = cv2.imread(img_name)\n    for cls in special_obj_cls:\n        if(not cls in classes):\n            continue\n        objs = os.listdir(SPEC_OBJ_DIR + cls)\n        for obj in objs:\n            print(obj)\n            if(not (obj.endswith(\'.png\') or obj.endswith(\'.jpg\'))):\n                continue\n            img_obj = cv2.imread(SPEC_OBJ_DIR + cls + \'/\' + obj)\n            dic_ = {\'name\':cls, \'difficult\':\'0\'}\n            obj_element = generate_xml_info_under_object(dic_)\n            new_pil_img, new_xml_etree = inset_obj_to_an_image_and_xml(img_obj, img, obj_element, tree)\n            pri

In [21]:
seed_xml_names = os.listdir(SEED_XML_DIR)
special_obj_cls = os.listdir(SPEC_OBJ_DIR)

classes = ['slagcar_10']

for cls in special_obj_cls:
        if(not cls in classes):
            continue
            
        objs = os.listdir(SPEC_OBJ_DIR + cls)
        for obj in objs:
            print(obj)
            if(not (obj.endswith('.png') or obj.endswith('.jpg'))):
                continue

            for xml_name in seed_xml_names:
                img_name = SEED_IMG_DIR + xml_name[:-3] + 'jpg'
                xml_name = SEED_XML_DIR + xml_name
                print(img_name)
                print(xml_name)
                tree=ET.parse(xml_name)
                img = cv2.imread(img_name)
                img_obj = cv2.imread(SPEC_OBJ_DIR + cls + '/' + obj)
                if((img.shape[0] < img_obj.shape[0]) or (img.shape[1] < img_obj.shape[1]) or 
                   ((img.shape[0]*img.shape[1])/(img_obj.shape[0]*img_obj.shape[1]) < 4)):
                    continue
                dic_ = {'name':cls, 'difficult':'0'}
                obj_element = generate_xml_info_under_object(dic_)
                new_pil_img, new_xml_etree = inset_obj_to_an_image_and_xml(img_obj, img, obj_element, tree)
                print('insert success')
                img = np.array(new_pil_img)

                ## save xml and img
                print('save image and xml')
                time_mark = datetime.datetime.now()
                time_str = time_mark.strftime("%Y%m%d%H%M%S_")
                new_pil_img = Image.fromarray(img[:,:,(2, 1, 0)])
                new_pil_img.save(GENE_IMG_WITH_SPECIAL_OBJ + time_str + img_name.split('/')[-1])
                new_xml_etree.write(GENE_XML_WITH_SPECIAL_OBJ + time_str + xml_name.split('/')[-1])
                #break

005616f3d4f6cbdb9f41f708f193cb38_0.jpg
/data/wxjs/JPEGImages/01c08702b7f7f562547e2a8718c8ab07.jpg
/data/wxjs/Changed_Annotations/01c08702b7f7f562547e2a8718c8ab07.xml
[602, 1381, 377, 1329]
insert success
save image and xml
/data/wxjs/JPEGImages/022014a454e80e353bef2690ea813d87.jpg
/data/wxjs/Changed_Annotations/022014a454e80e353bef2690ea813d87.xml
[1971, 2750, 808, 1760]
insert success
save image and xml
/data/wxjs/JPEGImages/014f328da8e0a69c217fc10635c3f7a8.jpg
/data/wxjs/Changed_Annotations/014f328da8e0a69c217fc10635c3f7a8.xml
[755, 1534, 985, 1937]
new_bbox not suitable, retry...
[663, 1442, 710, 1662]
new_bbox not suitable, retry...
[231, 1010, 1027, 1979]
new_bbox not suitable, retry...
[1856, 2635, 458, 1410]
insert success
save image and xml
/data/wxjs/JPEGImages/0127d83c870c04c6db0682ed6dd27162.jpg
/data/wxjs/Changed_Annotations/0127d83c870c04c6db0682ed6dd27162.xml
[811, 1590, 670, 1622]
new_bbox not suitable, retry...
[1113, 1892, 767, 1719]
new_bbox not suitable, retry...
[22

/data/wxjs/JPEGImages/023a381f2a1cf5ee8c11c1c9f0eadd2f.jpg
/data/wxjs/Changed_Annotations/023a381f2a1cf5ee8c11c1c9f0eadd2f.xml
[2174, 2953, 372, 1324]
new_bbox not suitable, retry...
[645, 1424, 975, 1927]
new_bbox not suitable, retry...
[407, 1186, 1009, 1961]
insert success
save image and xml
/data/wxjs/JPEGImages/0008ec851e66864a9ad38161f7e9d5a8.jpg
/data/wxjs/Changed_Annotations/0008ec851e66864a9ad38161f7e9d5a8.xml
[2009, 2788, 341, 1293]
new_bbox not suitable, retry...
[2541, 3320, 639, 1591]
new_bbox not suitable, retry...
[2547, 3326, 490, 1442]
new_bbox not suitable, retry...
[405, 1184, 184, 1136]
insert success
save image and xml
/data/wxjs/JPEGImages/0102e8b3e006c5b0146ec15cb4ea95a0.jpg
/data/wxjs/Changed_Annotations/0102e8b3e006c5b0146ec15cb4ea95a0.xml
[533, 1312, 100, 1052]
insert success
save image and xml
/data/wxjs/JPEGImages/021c507164d4d5e37cbae8cbac3199fd.jpg
/data/wxjs/Changed_Annotations/021c507164d4d5e37cbae8cbac3199fd.xml
[511, 1290, 508, 1460]
new_bbox not suita

/data/wxjs/JPEGImages/010237b9a95b6133bba49b4884ce2e42.jpg
/data/wxjs/Changed_Annotations/010237b9a95b6133bba49b4884ce2e42.xml
[707, 1486, 198, 1150]
insert success
save image and xml
/data/wxjs/JPEGImages/02af7fa274a3a49e9f9f3a8255eb6203.jpg
/data/wxjs/Changed_Annotations/02af7fa274a3a49e9f9f3a8255eb6203.xml
[681, 1460, 1158, 2110]
new_bbox not suitable, retry...
[462, 1241, 44, 996]
new_bbox not suitable, retry...
[248, 1027, 622, 1574]
insert success
save image and xml
/data/wxjs/JPEGImages/013643127506a43a01e303a725a8126f.jpg
/data/wxjs/Changed_Annotations/013643127506a43a01e303a725a8126f.xml
[1613, 2392, 205, 1157]
insert success
save image and xml
/data/wxjs/JPEGImages/01510218356609bbbbf72508a796fea5.jpg
/data/wxjs/Changed_Annotations/01510218356609bbbbf72508a796fea5.xml
[531, 1310, 777, 1729]
new_bbox not suitable, retry...
[173, 952, 880, 1832]
new_bbox not suitable, retry...
[1141, 1920, 906, 1858]
new_bbox not suitable, retry...
[1682, 2461, 762, 1714]
insert success
save im

/data/wxjs/JPEGImages/02e5e7e4dc66982c8e5a5498d3e1742d.jpg
/data/wxjs/Changed_Annotations/02e5e7e4dc66982c8e5a5498d3e1742d.xml
[889, 1668, 29, 981]
new_bbox not suitable, retry...
[1565, 2344, 905, 1857]
new_bbox not suitable, retry...
[1425, 2204, 564, 1516]
new_bbox not suitable, retry...
[1309, 2088, 742, 1694]
new_bbox not suitable, retry...
[647, 1426, 576, 1528]
insert success
save image and xml
026c62da4c7183cfd0b2dad62cb2ecde_0.jpg
/data/wxjs/JPEGImages/01c08702b7f7f562547e2a8718c8ab07.jpg
/data/wxjs/Changed_Annotations/01c08702b7f7f562547e2a8718c8ab07.xml
[1576, 2155, 991, 1928]
new_bbox not suitable, retry...
[2748, 3327, 532, 1469]
insert success
save image and xml
/data/wxjs/JPEGImages/022014a454e80e353bef2690ea813d87.jpg
/data/wxjs/Changed_Annotations/022014a454e80e353bef2690ea813d87.xml
[1736, 2315, 756, 1693]
insert success
save image and xml
/data/wxjs/JPEGImages/014f328da8e0a69c217fc10635c3f7a8.jpg
/data/wxjs/Changed_Annotations/014f328da8e0a69c217fc10635c3f7a8.xml
[11

/data/wxjs/JPEGImages/0102e8b3e006c5b0146ec15cb4ea95a0.jpg
/data/wxjs/Changed_Annotations/0102e8b3e006c5b0146ec15cb4ea95a0.xml
[2050, 2629, 399, 1336]
new_bbox not suitable, retry...
[2248, 2827, 205, 1142]
insert success
save image and xml
/data/wxjs/JPEGImages/021c507164d4d5e37cbae8cbac3199fd.jpg
/data/wxjs/Changed_Annotations/021c507164d4d5e37cbae8cbac3199fd.xml
[945, 1524, 1101, 2038]
insert success
save image and xml
/data/wxjs/JPEGImages/0190b4cb574472b6f9f5c0628f4cdd5c.jpg
/data/wxjs/Changed_Annotations/0190b4cb574472b6f9f5c0628f4cdd5c.xml
[1912, 2491, 960, 1897]
new_bbox not suitable, retry...
[853, 1432, 617, 1554]
insert success
save image and xml
/data/wxjs/JPEGImages/02acfbe1bcbb24461b3550c8bcb2d9d7.jpg
/data/wxjs/Changed_Annotations/02acfbe1bcbb24461b3550c8bcb2d9d7.xml
[1876, 2455, 900, 1837]
new_bbox not suitable, retry...
[593, 1172, 565, 1502]
insert success
save image and xml
/data/wxjs/JPEGImages/00046e49559722ee8e8159593d028888.jpg
/data/wxjs/Changed_Annotations/0004

...

/data/wxjs/JPEGImages/00b75476e165a156a73c29266dfdcb9e.jpg
/data/wxjs/Changed_Annotations/00b75476e165a156a73c29266dfdcb9e.xml
[1816, 2477, 329, 1130]
insert success
save image and xml
/data/wxjs/JPEGImages/024ba4df15d6edf6005f0d68ecbf0b99.jpg
/data/wxjs/Changed_Annotations/024ba4df15d6edf6005f0d68ecbf0b99.xml
[876, 1537, 127, 928]
new_bbox not suitable, retry...
[1987, 2648, 420, 1221]
insert success
save image and xml
/data/wxjs/JPEGImages/00b3d7c4884e52899ddce21fa5d4eea7.jpg
/data/wxjs/Changed_Annotations/00b3d7c4884e52899ddce21fa5d4eea7.xml
[2374, 3035, 774, 1575]
insert success
save image and xml
/data/wxjs/JPEGImages/0232a676ea70ec4533d2d397129da511.jpg
/data/wxjs/Changed_Annotations/0232a676ea70ec4533d2d397129da511.xml
[600, 1261, 140, 941]
insert success
save image and xml
/data/wxjs/JPEGImages/024e2a12a1e16f32256427efa4a299bd.jpg
/data/wxjs/Changed_Annotations/024e2a12a1e16f32256427efa4a299bd.xml
/data/wxjs/JPEGImages/00013ba74afbf27e0e41fd488be62c40.jpg
/data/wxjs/Changed_Ann

/data/wxjs/JPEGImages/00944c7a90932edf65e1c47f09270132.jpg
/data/wxjs/Changed_Annotations/00944c7a90932edf65e1c47f09270132.xml
[1880, 2541, 933, 1734]
new_bbox not suitable, retry...
[2662, 3323, 359, 1160]
insert success
save image and xml
/data/wxjs/JPEGImages/00acecb11ebad9b1c3647977e15b472b.jpg
/data/wxjs/Changed_Annotations/00acecb11ebad9b1c3647977e15b472b.xml
[616, 1277, 1135, 1936]
insert success
save image and xml
/data/wxjs/JPEGImages/0025e474100b4f05756e0df68619a0f0.jpg
/data/wxjs/Changed_Annotations/0025e474100b4f05756e0df68619a0f0.xml
[1378, 2039, 995, 1796]
new_bbox not suitable, retry...
[556, 1217, 778, 1579]
insert success
save image and xml
/data/wxjs/JPEGImages/00c47513ebb05513941987d5387d36ce.jpg
/data/wxjs/Changed_Annotations/00c47513ebb05513941987d5387d36ce.xml
[1801, 2462, 145, 946]
insert success
save image and xml
/data/wxjs/JPEGImages/006269facabc38b7ecd5243e286c67ae.jpg
/data/wxjs/Changed_Annotations/006269facabc38b7ecd5243e286c67ae.xml
/data/wxjs/JPEGImages/0

/data/wxjs/JPEGImages/00d5e6d41cb26afdfdca62ec3d0e88df.jpg
/data/wxjs/Changed_Annotations/00d5e6d41cb26afdfdca62ec3d0e88df.xml
[1106, 1767, 543, 1344]
new_bbox not suitable, retry...
[145, 806, 561, 1362]
insert success
save image and xml
/data/wxjs/JPEGImages/00a9e6f925d59563f493b282ed110cbc.jpg
/data/wxjs/Changed_Annotations/00a9e6f925d59563f493b282ed110cbc.xml
[752, 1413, 492, 1293]
insert success
save image and xml
/data/wxjs/JPEGImages/01b3abc94ea858bc2042b9d072279346.jpg
/data/wxjs/Changed_Annotations/01b3abc94ea858bc2042b9d072279346.xml
[522, 1183, 272, 1073]
new_bbox not suitable, retry...
[318, 979, 1100, 1901]
new_bbox not suitable, retry...
[815, 1476, 630, 1431]
new_bbox not suitable, retry...
[1443, 2104, 775, 1576]
insert success
save image and xml
/data/wxjs/JPEGImages/01f62baa787a82bf7585faf3ce82a8f4.jpg
/data/wxjs/Changed_Annotations/01f62baa787a82bf7585faf3ce82a8f4.xml
[891, 1552, 280, 1081]
new_bbox not suitable, retry...
[1944, 2605, 984, 1785]
insert success
save i

/data/wxjs/JPEGImages/01d84d68f98ba89a3ac6bcb676f7b00c.jpg
/data/wxjs/Changed_Annotations/01d84d68f98ba89a3ac6bcb676f7b00c.xml
[1029, 1389, 1141, 1650]
insert success
save image and xml
/data/wxjs/JPEGImages/01d9ad47ab619f0b0664e986ea790c06.jpg
/data/wxjs/Changed_Annotations/01d9ad47ab619f0b0664e986ea790c06.xml
[2071, 2431, 741, 1250]
insert success
save image and xml
/data/wxjs/JPEGImages/023a381f2a1cf5ee8c11c1c9f0eadd2f.jpg
/data/wxjs/Changed_Annotations/023a381f2a1cf5ee8c11c1c9f0eadd2f.xml
[303, 663, 687, 1196]
insert success
save image and xml
/data/wxjs/JPEGImages/0008ec851e66864a9ad38161f7e9d5a8.jpg
/data/wxjs/Changed_Annotations/0008ec851e66864a9ad38161f7e9d5a8.xml
[1675, 2035, 1067, 1576]
new_bbox not suitable, retry...
[2406, 2766, 774, 1283]
new_bbox not suitable, retry...
[885, 1245, 1058, 1567]
insert success
save image and xml
/data/wxjs/JPEGImages/0102e8b3e006c5b0146ec15cb4ea95a0.jpg
/data/wxjs/Changed_Annotations/0102e8b3e006c5b0146ec15cb4ea95a0.xml
[2693, 3053, 364, 873

/data/wxjs/JPEGImages/0221579b3252d9bebb81798cf8543809.jpg
/data/wxjs/Changed_Annotations/0221579b3252d9bebb81798cf8543809.xml
[2706, 3066, 1292, 1801]
insert success
save image and xml
/data/wxjs/JPEGImages/022d0841346351c4c8f767242d6d5456.jpg
/data/wxjs/Changed_Annotations/022d0841346351c4c8f767242d6d5456.xml
[2665, 3025, 876, 1385]
insert success
save image and xml
/data/wxjs/JPEGImages/01017b4f5dc5d635199d9670afd949fe.jpg
/data/wxjs/Changed_Annotations/01017b4f5dc5d635199d9670afd949fe.xml
[1588, 1948, 620, 1129]
insert success
save image and xml
/data/wxjs/JPEGImages/01c672bdb18dc60de44be911d0f5a6a2.jpg
/data/wxjs/Changed_Annotations/01c672bdb18dc60de44be911d0f5a6a2.xml
[2698, 3058, 96, 605]
insert success
save image and xml
/data/wxjs/JPEGImages/025017ce46d063cb1f5db57fe2e48990.jpg
/data/wxjs/Changed_Annotations/025017ce46d063cb1f5db57fe2e48990.xml
[1815, 2175, 415, 924]
insert success
save image and xml
/data/wxjs/JPEGImages/0230f60cfb5c626ea94eb4f3b0541e9c.jpg
/data/wxjs/Changed